In [6]:
from pyspark.sql import SparkSession
import hashlib

spark = SparkSession.builder.appName("Lab4").getOrCreate()

In [7]:
import json

reviews_raw = spark.sparkContext.textFile('movies 1.json')
reviews = reviews_raw.map(lambda line: json.loads(line))

In [8]:
def get_hash(s):
    return int(hashlib.sha1(s).hexdigest(), 16) % (10 ** 8)

ratings = reviews.map(lambda entry: (
    get_hash(entry['user_id'].encode('utf-8')),
    get_hash(entry['product_id'].encode('utf-8')),
    int(entry['score'])
))
train_data = ratings.filter(lambda entry: ((entry[0] + entry[1]) % 10) >= 2)
test_data = ratings.filter(lambda entry: ((entry[0] + entry[1]) % 10) < 2)

print ("Number of train samples: " + str(train_data.count()))
print ("Number of test samples: " + str(test_data.count()))

Number of train samples: 39992
Number of test samples: 10008


In [9]:
from pyspark.mllib.recommendation import ALS

rank = 20
numIterations = 20

model = ALS.train(train_data, rank, numIterations)

unknown = test_data.map(lambda entry: (int(entry[0]), int(entry[1])))

predictions = model.predictAll(unknown).map(lambda r: ((int(r[0]), int(r[1])), r[2]))

In [10]:
true_and_predictions = test_data.map(lambda r: ((int(r[0]), int(r[1])), r[2])).join(predictions)

MSE = true_and_predictions.map(lambda r: (r[1][0] - r[1][1]) ** 2).mean()

print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 7.779685460451557
